<a href="https://colab.research.google.com/github/yms0606/SecuLayer/blob/main/%EC%9B%94%EA%B0%84%20%EB%8D%B0%EC%9D%B4%EC%BD%98%20%ED%95%AD%EA%B3%B5%ED%8E%B8%20%EC%A7%80%EC%97%B0%20%EC%98%88%EC%B8%A1%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/%EC%9B%94%EA%B0%84_%EB%8D%B0%EC%9D%B4%EC%BD%98_%ED%95%AD%EA%B3%B5%ED%8E%B8_%EC%A7%80%EC%97%B0_%EC%98%88%EC%B8%A1_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import os
import gc

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

In [ ]:
def csv_to_parquet(csv_path, save_name):
  df = pd.read_csv(csv_path)
  df.to_parquet(f'{save_name}.parquet')
  del df
  gc.collect()

csv_to_parquet('/content/drive/MyDrive/ML study/delay_airplane/train.csv', 'train')
csv_to_parquet('/content/drive/MyDrive/ML study/delay_airplane/test.csv', 'test')

In [ ]:
train = pd.read_parquet('/content/train.parquet')
test = pd.read_parquet('/content/test.parquet')

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/ML study/delay_airplane/sample_submission.csv')

In [ ]:
train = train.drop(['ID','Cancelled','Diverted','Carrier_Code(IATA)'], axis = 1)
test = test.drop(['ID','Cancelled','Diverted','Carrier_Code(IATA)'], axis = 1)

In [ ]:
#temp = train[['Origin_Airport_ID', 'Distance']]
#temp_test = test[['Origin_Airport_ID', 'Distance']]
temp = train.groupby(["Origin_Airport_ID", "Distance"])[["Origin_State", "Destination_State"]].agg(pd.Series.mode)
temp = temp.reset_index()

temp_test = test.groupby(["Origin_Airport_ID", "Distance"])[["Origin_State", "Destination_State"]].agg(pd.Series.mode)
temp_test = temp_test.reset_index()

# Airport_ID와 Distance의 조합에 따른 State들의 최빈값

In [ ]:
temp

,Origin_Airport_ID,Distance,Origin_State,Destination_State
0,10135,1018.0,Pennsylvania,Florida
1,10135,1041.0,Pennsylvania,Florida
2,10135,425.0,Pennsylvania,Michigan
3,10135,481.0,Pennsylvania,North Carolina
4,10135,518.0,Pennsylvania,South Carolina
...,...,...,...,...
6625,16101,103.0,Washington,Washington
6626,16218,1022.0,Arizona,Texas
6627,16218,160.0,Arizona,Arizona
6628,16869,553.0,North Dakota,Minnesota


In [ ]:
temp = temp.astype({'Origin_Airport_ID':'str'})
temp = temp.astype({'Distance':'str'})

In [ ]:
temp_test = temp_test.astype({'Origin_Airport_ID':'str'})
temp_test = temp_test.astype({'Distance':'str'})

In [ ]:
temp['agg'] = temp['Origin_Airport_ID']+temp['Distance']
temp = temp.drop(['Origin_Airport_ID','Distance'], axis=1)

temp_test['agg'] = temp_test['Origin_Airport_ID']+temp_test['Distance']
temp_test = temp_test.drop(['Origin_Airport_ID','Distance'], axis=1)

In [ ]:
train = train.astype({'Origin_Airport_ID': 'str'})
train = train.astype({'Distance': 'str'})
train['agg'] = train['Origin_Airport_ID'] + train['Distance']

test = test.astype({'Origin_Airport_ID': 'str'})
test = test.astype({'Distance': 'str'})
test['agg'] = test['Origin_Airport_ID'] + test['Distance']

In [ ]:
train

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_ID(DOT),Tail_Number,Delay,agg
0,4,15,NaN,NaN,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,19393.0,N7858A,None,13851419.0
1,8,15,740.0,1024.0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,20304.0,N125SY,None,139301250.0
2,9,6,1610.0,1805.0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,19805.0,N103US,None,11057544.0
3,7,10,905.0,1735.0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,NaN,N595UA,None,128922454.0
4,1,11,900.0,1019.0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,20304.0,N161SY,None,14771250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,9,18,936.0,1243.0,ORD,13930,None,PHL,14100,None,678.0,United Air Lines Inc.,19977.0,N477UA,None,13930678.0
999996,5,30,920.0,1028.0,FAR,11637,None,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,NaN,N439SW,None,11637223.0
999997,6,28,800.0,1340.0,OAK,13796,None,HOU,12191,Texas,1642.0,Southwest Airlines Co.,19393.0,N230WN,None,137961642.0
999998,9,27,1613.0,1824.0,BNA,10693,Tennessee,ATL,10397,None,214.0,Delta Air Lines Inc.,19790.0,N968DL,None,10693214.0


In [ ]:
temp

,Origin_State,Destination_State,agg
0,Pennsylvania,Florida,101351018.0
1,Pennsylvania,Florida,101351041.0
2,Pennsylvania,Michigan,10135425.0
3,Pennsylvania,North Carolina,10135481.0
4,Pennsylvania,South Carolina,10135518.0
...,...,...,...
6625,Washington,Washington,16101103.0
6626,Arizona,Texas,162181022.0
6627,Arizona,Arizona,16218160.0
6628,North Dakota,Minnesota,16869553.0


In [ ]:
train = pd.merge(left=train, right=temp, how='left',on=['agg'])
test = pd.merge(left=test, right=temp_test, how='left',on=['agg'])

In [ ]:
train = train.drop(['Origin_State_x','Destination_State_x','agg'],axis=1)
train = train.astype({'Origin_Airport_ID':'int64'})
train = train.astype({'Distance':'float64'})

test = test.drop(['Origin_State_x','Destination_State_x','agg'],axis=1)
test = test.astype({'Origin_Airport_ID':'int64'})
test = test.astype({'Distance':'float64'})

In [ ]:
train = train.rename(columns = {'Origin_State_y':'Origin_State', 'Destination_State_y':'Destination_State'})
test = test.rename(columns = {'Origin_State_y':'Origin_State', 'Destination_State_y':'Destination_State'})

In [ ]:
NaN_col = ['Airline','Estimated_Departure_Time','Estimated_Arrival_Time','Carrier_ID(DOT)']

for col in NaN_col:
  mode = train[col].mode()[0]
  train[col] = train[col].fillna(mode)

  if col in test.columns:
    test[col] = test[col].fillna(mode)

In [ ]:
train['Delay'].unique()

array([None, 'Not_Delayed', 'Delayed'], dtype=object)

In [ ]:
qual_col = ['Origin_Airport','Origin_State','Destination_Airport','Destination_State','Airline','Tail_Number']

for ord in qual_col:
  le = LabelEncoder()
  le.fit(train[ord].astype(str))
  train[ord] = le.transform(train[ord].astype(str))

  prev_class = list(le.classes_)

  for label in np.unique(test[ord].astype(str)):
    if label not in prev_class:
      prev_class.append(label)

  le.classes_ = np.array(prev_class)
  test[ord] = le.transform(test[ord].astype(str))


In [ ]:
train_unlabel = train[train['Delay'].isna()].reset_index(drop=True)
train_label = train[train['Delay'].notna()].reset_index(drop=True)

In [ ]:
train_label

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport,Origin_Airport_ID,Destination_Airport,Destination_Airport_ID,Distance,Airline,Carrier_ID(DOT),Tail_Number,Delay,Origin_State,Destination_State
0,4,13,1545.0,1900.0,119,11618,93,11278,199.0,21,20452.0,3435,Not_Delayed,28,47
1,1,20,1742.0,1903.0,119,11618,47,10721,200.0,26,19393.0,3495,Not_Delayed,28,19
2,6,13,1420.0,1550.0,59,10821,74,11057,361.0,23,19393.0,4083,Not_Delayed,18,31
3,8,13,1730.0,1844.0,93,11278,277,14122,204.0,21,19393.0,241,Delayed,47,36
4,1,12,1015.0,1145.0,72,11042,94,11292,1201.0,23,19393.0,5171,Not_Delayed,33,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254996,10,11,600.0,2003.0,310,14683,256,13930,1041.0,22,20304.0,488,Not_Delayed,42,11
254997,5,2,1759.0,1926.0,204,12953,93,11278,214.0,23,20452.0,5204,Delayed,30,47
254998,10,10,940.0,1056.0,223,13256,169,12266,316.0,19,20378.0,5350,Delayed,42,42
254999,8,8,1914.0,2039.0,296,14492,183,12451,407.0,14,20436.0,1499,Not_Delayed,31,7


In [ ]:
train_label['Delay'] = train_label['Delay'].map(lambda x:0 if x=='Not_Delayed' else 1)

In [ ]:
y = train_label['Delay']
X = train_label.drop(['Delay'], axis =1)

In [ ]:
y.unique()

array([0, 1])

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
xgb = XGBClassifier(tree_method='gpu_hist')
param = {
    'n_estimators' : range(100,1000,100),
    'learning_rate' : [0.01,0.03,0.05,0.07,0.1,0.2,0.5],
    'max_depth' : range(3,15,1)
}

grid = GridSearchCV(xgb, param_grid = param, scoring='accuracy', cv=5)
grid.fit(X,y)

print(grid.best_estimator_)

In [ ]:
xgb = XGBClassifier(n_jobs=-1, scale_pos_weight=4.6, max_depth=3, random_state=2)
xgb.fit(X, y)

In [ ]:
y_pred = xgb.predict_proba(test)
y_pred

array([[0.52210855, 0.47789147],
       [0.47249502, 0.527505  ],
       [0.35724545, 0.64275455],
       ...,
       [0.40108633, 0.59891367],
       [0.44847763, 0.5515224 ],
       [0.5685291 , 0.43147084]], dtype=float32)

In [ ]:
submit

,ID,Not_Delayed,Delayed
0,TEST_000000,0,1
1,TEST_000001,0,1
2,TEST_000002,0,1
3,TEST_000003,0,1
4,TEST_000004,0,1
...,...,...,...
999995,TEST_999995,0,1
999996,TEST_999996,0,1
999997,TEST_999997,0,1
999998,TEST_999998,0,1


In [ ]:
submission = pd.DataFrame(data=y_pred, columns=[['Not_Delayed','Delayed']], index=submit.index)
submission

,Not_Delayed,Delayed
0,0.522109,0.477891
1,0.472495,0.527505
2,0.357245,0.642755
3,0.317087,0.682913
4,0.284292,0.715708
...,...,...
999995,0.377565,0.622435
999996,0.616197,0.383803
999997,0.401086,0.598914
999998,0.448478,0.551522


In [ ]:
submit['Not_Delayed'] = submission['Not_Delayed']
submit['Delayed'] = submission['Delayed']
submit

,ID,Not_Delayed,Delayed
0,TEST_000000,0.522109,0.477891
1,TEST_000001,0.472495,0.527505
2,TEST_000002,0.357245,0.642755
3,TEST_000003,0.317087,0.682913
4,TEST_000004,0.284292,0.715708
...,...,...,...
999995,TEST_999995,0.377565,0.622435
999996,TEST_999996,0.616197,0.383803
999997,TEST_999997,0.401086,0.598914
999998,TEST_999998,0.448478,0.551522


In [ ]:
submit.to_csv('submission1.csv',index=False)